# Solution: Image Data Augmentation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/computer-vision-with-embedded-machine-learning/blob/master/2.3.5%20-%20Project%20-%20Data%20Augmentation/solution_image_data_augmentation.ipynb)

This is an example for creating an augmented dataset. It will transform input images to create a series of augmented samples that are saved in a new output directory.

Create a folder named "dataset" in the /content directory and upload your images there. The images should be divided into their respective classes, where each class has its own folder with the name of the class. For example:

<pre>
/content
    |- dataset
        |- background
        |- capacitor
        |- diode
        |- led
        |- resistor
</pre>

The original images along with their transforms will be saved in the output directory. Each output file will be the original filename appended with "_{num}" where {num} is some incrementing value based on the total number of transforms performed per image.

For example, if you have a file named "0.png" in /content/dataset/resistor, it will become "0_0.png" in /content/output/resistor. The first transform will be "0_1.png", the second transform will be "0_2.png" and so on.

Run each of the cells paying attention to their contents and output. Fill out the necessary parts of the functions where you find the following comment:

```
# >>> ENTER YOUR CODE HERE <<<
```

Author: EdgeImpulse, Inc.<br>
Date: August 3, 2021<br>
License: [Apache-2.0](apache.org/licenses/LICENSE-2.0)<br>

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import PIL

import skimage.transform
import skimage.util

In [30]:
cd bees

C:\Users\31653\Desktop\Prive\insecten\bees


In [31]:
import glob
bees = glob.glob('*.jpg')
bees

['1092977343_cb42b38d62.jpg',
 '1093831624_fb5fbe2308.jpg',
 '1097045929_1753d1c765.jpg',
 '1232245714_f862fbe385.jpg',
 '129236073_0985e91c7d.jpg',
 '1295655112_7813f37d21.jpg',
 '132511197_0b86ad0fff.jpg',
 '132826773_dbbcb117b9.jpg',
 '150013791_969d9a968b.jpg',
 '1508176360_2972117c9d.jpg',
 '154600396_53e1252e52.jpg',
 '16838648_415acd9e3f.jpg',
 '1691282715_0addfdf5e8.jpg',
 '17209602_fe5a5a746f.jpg',
 '174142798_e5ad6d76e0.jpg',
 '1799726602_8580867f71.jpg',
 '1807583459_4fe92b3133.jpg',
 '196430254_46bd129ae7.jpg',
 '196658222_3fffd79c67.jpg',
 '198508668_97d818b6c4.jpg',
 '2031225713_50ed499635.jpg',
 '2037437624_2d7bce461f.jpg',
 '2053200300_8911ef438a.jpg',
 '205835650_e6f2614bee.jpg',
 '208702903_42fb4d9748.jpg',
 '21399619_3e61e5bb6f.jpg',
 '2227611847_ec72d40403.jpg',
 '2321139806_d73d899e66.jpg',
 '2330918208_8074770c20.jpg',
 '2345177635_caf07159b3.jpg',
 '2358061370_9daabbd9ac.jpg',
 '2364597044_3c3e3fc391.jpg',
 '2384149906_2cd8b0b699.jpg',
 '2397446847_04ef3cd3e1.jpg

In [27]:
import glob
my_list = glob.glob('Users/31653/Desktop/Prive/insecten/bees/*.jpg')
my_list

[]

In [34]:
### Settings

# Location of dataset and output folder
DATASET_PATH = "insecten"
OUT_PATH = "/bees"
OUT_ZIP = "augmented_dataset.zip"

# File format to use for new dataset
IMG_EXT = ".png"

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)

In [36]:
### Create output directory
try:
  os.makedirs(OUT_PATH)
  print('done')
except FileExistsError:
  print("WARNING: Output directory already exists. Check to make sure it is empty.")

## Transform Functions

Create one or more functions that transform an input image.

In [39]:
### Example: Function to create 3 new flipped images of the input
def create_flipped(img):

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped

In [53]:
from PIL import Image #import PIL.Image 
img = Image.open('1092977343_cb42b38d62.jpg')

img_ar = np.asarray(img)
flipped = np.fliplr(img_ar)
flip_im= Image.fromarray(flipped)
data.save('flipped.png')
data.show()

In [ ]:
### Function to create new rotated images of the input
def create_rotated(img, rotations):

  # Create list of rotated images (keep 8-bit values)
  rotated = []
  for rot in rotations:
    img_rot = skimage.transform.rotate(img, angle=rot, mode='edge', preserve_range=True)
    img_rot = img_rot.astype(np.uint8)
    rotated.append(img_rot)

  return rotated

In [ ]:
### Function to create random scale/crop (zoom) images
def create_random_zooms(img, scale_factor, num_crops):

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create scaled images (e.g. make the image bigger) and keep 8-bit values
  img_scaled = skimage.transform.rescale(img, 
                                        scale=scale_factor, 
                                        anti_aliasing=True, 
                                        multichannel=True,
                                        preserve_range=True)
  img_scaled = img_scaled.astype(np.uint8)

  # Get height and width of scaled image
  s_h = img_scaled.shape[0]
  s_w = img_scaled.shape[1]

  # Create list of random zooms
  zooms = []
  for i in range(num_crops):
    
    # Randomly choose start of crop point
    crop_y = round(random.random() * (s_h - height))
    crop_x = round(random.random() * (s_w - width))

    # Crop scaled image
    zoom = img_scaled[crop_y:(crop_y + height), crop_x:(crop_x + width), :]

    # Append zoomed image to list
    zooms.append(zoom)

  return zooms

In [ ]:
### Function to create a random set of translated images (no more than 1/4 of width or height away)
def create_random_translations(img, num_translations):

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create list of random translations
  translations = []
  for i in range(num_translations):
  
    # Choose random amount to translate (up to 1/4 image width, height) in either direction
    tr_y = round((0.5 - random.random()) * (height / 2))
    tr_x = round((0.5 - random.random()) * (width / 2))

    # Perform translation to create new image
    translation = skimage.transform.AffineTransform(translation=(tr_y, tr_x))
    img_tr = skimage.transform.warp(img, translation, mode='edge', preserve_range=True)
    img_tr = img_tr.astype(np.uint8)

    # Append translated image to list
    translations.append(img_tr)

  return translations

In [ ]:
### Function to add random noise to images
def create_noisy(img, types, seed=None):

  # Add noise of different types
  noisy_imgs = []
  for t in types:
    noise = skimage.util.random_noise(img, mode=t, seed=seed)
    noise = (noise * 255).astype(np.uint8)
    noisy_imgs.append(noise)

  return noisy_imgs

## Perform Transforms

Call your functions to create a set of augmented data.

In [ ]:
### Function to open image and create a list of new transforms
# NOTE: You will need to call your functions here!
def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Perform transforms (call your functions)
  img_tfs.append(create_flipped(img_array))
  # >>> ENTER YOUR CODE HERE <<<
  # e.g. img_tfs.append(create_translations(img_array, 2))
  img_tfs.append(create_flipped(img_array))
  img_tfs.append(create_rotated(img_array, [45, 90, 135]))
  img_tfs.append(create_random_zooms(img_array, 1.3, 2))
  img_tfs.append(create_random_translations(img_array, 2))
  img_tfs.append(create_noisy(img_array, ['gaussian', 's&p'], SEED))

  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs

In [ ]:
### Load all images, create transforms, and save in output directory

# Find the directories in the dataset folder (skip the Jupyter Notebook checkpoints hidden folder)
for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if os.path.isdir(class_dir) and label != ".ipynb_checkpoints":

    # Create output directory
    out_path = os.path.join(OUT_PATH, label)
    os.makedirs(out_path, exist_ok=True)

    # Go through each image in the subfolder
    for i, filename in enumerate(os.listdir(class_dir)):

      # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
      if filename != ".ipynb_checkpoints":

        # Get the root of the filename before the extension
        file_root = os.path.splitext(filename)[0]

        # Do all transforms for that one image
        file_path = os.path.join(DATASET_PATH, label, filename)
        img_tfs = create_transforms(file_path)

        # Save images to new files in output directory
        for i, img in enumerate(img_tfs):

          # Create a Pillow image from the Numpy array
          img_pil = PIL.Image.fromarray(img)

          # Construct filename (<orignal>_<transform_num>.<EXT>)
          out_file_path = os.path.join(out_path, file_root + "_" + str(i) + IMG_EXT)

          # Convert Numpy array to image and save as a file
          img_pil = PIL.Image.fromarray(img)
          img_pil.save(out_file_path)

In [ ]:
### Zip our new dataset (use '!' to call Linux commands)
!zip -r -q "{OUT_ZIP}" "{OUT_PATH}"